In [8]:
import sys
from pathlib import Path

ROOT = Path.cwd().parent  # volta de notebooks/ para a raiz (jarvis-planejador-financeiro)
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from database import init_database, get_connection

In [9]:
from database import init_database, get_connection

# Garante que o banco e as tabelas existam
init_database()

with get_connection() as conn:
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;").fetchall()

tables


In [10]:
from pprint import pprint

with get_connection() as conn:
    schema = {}
    for (name,) in tables:
        schema[name] = conn.execute(f"SELECT sql FROM sqlite_master WHERE type='table' AND name='{name}';").fetchone()[0]

pprint(schema)


{'categories': 'CREATE TABLE categories (\n'
               '                category_id INTEGER PRIMARY KEY '
               'AUTOINCREMENT,\n'
               '                user_phone TEXT,\n'
               '                category_name TEXT,\n'
               '                description TEXT,\n'
               '                created_at DATETIME DEFAULT '
               'CURRENT_TIMESTAMP,\n'
               '                FOREIGN KEY (user_phone) REFERENCES '
               'users(user_phone)\n'
               '            )',
 'sqlite_sequence': 'CREATE TABLE sqlite_sequence(name,seq)',
 'transactions': 'CREATE TABLE transactions (\n'
                 '                transaction_id INTEGER PRIMARY KEY '
                 'AUTOINCREMENT,\n'
                 '                user_phone TEXT,\n'
                 '                category_id INTEGER,\n'
                 '                amount REAL,\n'
                 '                expense_description TEXT,\n'
             

## Visualização dos Schemas

Imprimindo de forma mais legível cada tabela:


In [11]:
with get_connection() as conn:
    for (table_name,) in tables:
        if table_name == 'sqlite_sequence':  # pula tabela interna
            continue
        
        print(f"\n{'='*60}")
        print(f"Tabela: {table_name.upper()}")
        print('='*60)
        
        # Pega colunas
        columns = conn.execute(f"PRAGMA table_info({table_name})").fetchall()
        
        print("\nColunas:")
        for col in columns:
            col_dict = dict(col)
            pk = " [PK]" if col_dict['pk'] else ""
            notnull = " NOT NULL" if col_dict['notnull'] else ""
            default = f" DEFAULT {col_dict['dflt_value']}" if col_dict['dflt_value'] else ""
            print(f"  • {col_dict['name']}: {col_dict['type']}{pk}{notnull}{default}")
        
        # Pega foreign keys
        fks = conn.execute(f"PRAGMA foreign_key_list({table_name})").fetchall()
        if fks:
            print("\nChaves Estrangeiras:")
            for fk in fks:
                fk_dict = dict(fk)
                print(f"  • {fk_dict['from']} → {fk_dict['table']}({fk_dict['to']})")




Tabela: CATEGORIES

Colunas:
  • category_id: INTEGER [PK]
  • user_phone: TEXT
  • category_name: TEXT
  • description: TEXT
  • created_at: DATETIME DEFAULT CURRENT_TIMESTAMP

Chaves Estrangeiras:
  • user_phone → users(user_phone)

Tabela: TRANSACTIONS

Colunas:
  • transaction_id: INTEGER [PK]
  • user_phone: TEXT
  • category_id: INTEGER
  • amount: REAL
  • expense_description: TEXT
  • created_at: DATETIME DEFAULT CURRENT_TIMESTAMP

Chaves Estrangeiras:
  • category_id → categories(category_id)
  • user_phone → users(user_phone)

Tabela: USER_RULES

Colunas:
  • rule_id: INTEGER [PK]
  • user_phone: TEXT
  • category_id: INTEGER
  • period_type: TEXT DEFAULT 'mensal'
  • period_start: DATETIME DEFAULT CURRENT_TIMESTAMP
  • period_end: DATETIME
  • limit_value: REAL
  • current_total: REAL DEFAULT 0
  • last_updated: DATETIME
  • active: INTEGER DEFAULT 1

Chaves Estrangeiras:
  • category_id → categories(category_id)
  • user_phone → users(user_phone)

Tabela: USERS

Colunas:
 

## Criando Dados de Teste (Dummy Data)

Vamos popular o banco com dados fictícios para demonstração:


In [12]:
from datetime import datetime, timedelta

with get_connection() as conn:
    # 1. Inserir usuários
    usuarios = [
        ("5511999999999", "João Silva"),
        ("5511888888888", "Maria Santos"),
        ("5511777777777", "Pedro Costa")
    ]
    
    for phone, name in usuarios:
        conn.execute(
            "INSERT OR IGNORE INTO users (user_phone, user_name, created_at, last_message_at) VALUES (?, ?, ?, ?)",
            (phone, name, datetime.now(), datetime.now())
        )
    
    # 2. Inserir categorias para cada usuário
    categorias = [
        ("5511999999999", "Alimentação", "Gastos com comida e bebida"),
        ("5511999999999", "Transporte", "Uber, ônibus, combustível"),
        ("5511999999999", "Lazer", "Cinema, festas, hobbies"),
        ("5511888888888", "Mercado", "Compras de supermercado"),
        ("5511888888888", "Saúde", "Farmácia e consultas"),
        ("5511777777777", "Educação", "Livros e cursos"),
    ]
    
    for phone, cat_name, desc in categorias:
        conn.execute(
            "INSERT OR IGNORE INTO categories (user_phone, category_name, description) VALUES (?, ?, ?)",
            (phone, cat_name, desc)
        )
    
    # 3. Pegar IDs das categorias criadas
    cat_alimentacao = conn.execute(
        "SELECT category_id FROM categories WHERE user_phone=? AND category_name=?",
        ("5511999999999", "Alimentação")
    ).fetchone()[0]
    
    cat_transporte = conn.execute(
        "SELECT category_id FROM categories WHERE user_phone=? AND category_name=?",
        ("5511999999999", "Transporte")
    ).fetchone()[0]
    
    cat_lazer = conn.execute(
        "SELECT category_id FROM categories WHERE user_phone=? AND category_name=?",
        ("5511999999999", "Lazer")
    ).fetchone()[0]
    
    # 4. Inserir transações
    transacoes = [
        ("5511999999999", cat_alimentacao, 45.90, "Almoço no restaurante", datetime.now() - timedelta(days=2)),
        ("5511999999999", cat_alimentacao, 120.50, "Compras no mercado", datetime.now() - timedelta(days=1)),
        ("5511999999999", cat_transporte, 25.00, "Uber para o trabalho", datetime.now() - timedelta(hours=5)),
        ("5511999999999", cat_transporte, 18.50, "Uber voltando", datetime.now() - timedelta(hours=2)),
        ("5511999999999", cat_lazer, 80.00, "Cinema com amigos", datetime.now() - timedelta(days=3)),
    ]
    
    for phone, cat_id, amount, desc, created in transacoes:
        conn.execute(
            "INSERT INTO transactions (user_phone, category_id, amount, expense_description, created_at) VALUES (?, ?, ?, ?, ?)",
            (phone, cat_id, amount, desc, created)
        )
    
    # 5. Inserir regras de limite
    regras = [
        ("5511999999999", cat_alimentacao, "mensal", 500.00),
        ("5511999999999", cat_transporte, "semanal", 150.00),
        ("5511999999999", cat_lazer, "mensal", 300.00),
    ]
    
    for phone, cat_id, period, limit_val in regras:
        # Calcula total atual
        total = conn.execute(
            "SELECT COALESCE(SUM(amount), 0) FROM transactions WHERE user_phone=? AND category_id=?",
            (phone, cat_id)
        ).fetchone()[0]
        
        conn.execute(
            """INSERT INTO user_rules 
               (user_phone, category_id, period_type, limit_value, current_total, last_updated, active) 
               VALUES (?, ?, ?, ?, ?, ?, 1)""",
            (phone, cat_id, period, limit_val, total, datetime.now())
        )
    
    conn.commit()

print("✅ Dados de teste inseridos com sucesso!")


✅ Dados de teste inseridos com sucesso!


/var/folders/_k/ygqs0kfx3h58dkljhd410f9c0000gn/T/ipykernel_24889/253094500.py:12: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  conn.execute(
/var/folders/_k/ygqs0kfx3h58dkljhd410f9c0000gn/T/ipykernel_24889/253094500.py:59: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  conn.execute(
/var/folders/_k/ygqs0kfx3h58dkljhd410f9c0000gn/T/ipykernel_24889/253094500.py:78: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  conn.execute(


## Visualizando os Dados Inseridos

Imprimindo o conteúdo de cada tabela:


In [13]:
import pandas as pd

with get_connection() as conn:
    print("=" * 80)
    print("TABELA: USERS")
    print("=" * 80)
    df_users = pd.read_sql_query("SELECT * FROM users", conn)
    display(df_users)
    
    print("\n" + "=" * 80)
    print("TABELA: CATEGORIES")
    print("=" * 80)
    df_categories = pd.read_sql_query("SELECT * FROM categories", conn)
    display(df_categories)
    
    print("\n" + "=" * 80)
    print("TABELA: TRANSACTIONS")
    print("=" * 80)
    df_transactions = pd.read_sql_query("""
        SELECT 
            t.transaction_id,
            t.user_phone,
            c.category_name,
            t.amount,
            t.expense_description,
            t.created_at
        FROM transactions t
        LEFT JOIN categories c ON t.category_id = c.category_id
        ORDER BY t.created_at DESC
    """, conn)
    display(df_transactions)
    
    print("\n" + "=" * 80)
    print("TABELA: USER_RULES")
    print("=" * 80)
    df_rules = pd.read_sql_query("""
        SELECT 
            r.rule_id,
            r.user_phone,
            c.category_name,
            r.period_type,
            r.limit_value,
            r.current_total,
            r.active
        FROM user_rules r
        LEFT JOIN categories c ON r.category_id = c.category_id
    """, conn)
    display(df_rules)


TABELA: USERS


,user_phone,user_name,created_at,last_message_at,days_inactive,notify_opt_in,notify_hour
0,5511999999999,None,2025-11-12 11:35:30,2025-11-12 11:35:30,0,1,22
1,5511888888888,None,2025-11-12 11:35:30,2025-11-12 11:35:30,0,1,22
2,5511777777777,Pedro Costa,2025-11-12 20:05:35.269905,2025-11-12 20:05:35.269905,0,1,22



TABELA: CATEGORIES


,category_id,user_phone,category_name,description,created_at
0,1,5511999999999,Alimentação,Gastos com comida e bebida,2025-11-12 23:05:35
1,2,5511999999999,Transporte,"Uber, ônibus, combustível",2025-11-12 23:05:35
2,3,5511999999999,Lazer,"Cinema, festas, hobbies",2025-11-12 23:05:35
3,4,5511888888888,Mercado,Compras de supermercado,2025-11-12 23:05:35
4,5,5511888888888,Saúde,Farmácia e consultas,2025-11-12 23:05:35
5,6,5511777777777,Educação,Livros e cursos,2025-11-12 23:05:35



TABELA: TRANSACTIONS


,transaction_id,user_phone,category_name,amount,expense_description,created_at
0,7,5511999999999,Transporte,18.5,Uber voltando,2025-11-12 18:05:35.270697
1,6,5511999999999,Transporte,25.0,Uber para o trabalho,2025-11-12 15:05:35.270695
2,1,5511999999999,None,0.0,Gastei 40 reais no mercado,2025-11-12 11:35:30
3,2,5511999999999,None,0.0,"Uber residência → trabalho 25,90",2025-11-12 11:35:30
4,3,5511888888888,None,0.0,"Cinema 50,00",2025-11-12 11:35:30
5,5,5511999999999,Alimentação,120.5,Compras no mercado,2025-11-11 20:05:35.270694
6,4,5511999999999,Alimentação,45.9,Almoço no restaurante,2025-11-10 20:05:35.270688
7,8,5511999999999,Lazer,80.0,Cinema com amigos,2025-11-09 20:05:35.270698



TABELA: USER_RULES


,rule_id,user_phone,category_name,period_type,limit_value,current_total,active
0,1,5511999999999,Alimentação,mensal,500.0,166.4,1
1,2,5511999999999,Transporte,semanal,150.0,43.5,1
2,3,5511999999999,Lazer,mensal,300.0,80.0,1


## Resumo Analítico

Estatísticas dos dados inseridos:


In [14]:
with get_connection() as conn:
    # Total de registros
    print("📊 ESTATÍSTICAS GERAIS")
    print("-" * 60)
    total_users = conn.execute("SELECT COUNT(*) FROM users").fetchone()[0]
    total_categories = conn.execute("SELECT COUNT(*) FROM categories").fetchone()[0]
    total_transactions = conn.execute("SELECT COUNT(*) FROM transactions").fetchone()[0]
    total_rules = conn.execute("SELECT COUNT(*) FROM user_rules").fetchone()[0]
    
    print(f"Total de usuários: {total_users}")
    print(f"Total de categorias: {total_categories}")
    print(f"Total de transações: {total_transactions}")
    print(f"Total de regras: {total_rules}")
    
    # Gastos por categoria
    print("\n💰 GASTOS POR CATEGORIA (João Silva)")
    print("-" * 60)
    gastos = conn.execute("""
        SELECT 
            c.category_name,
            COUNT(t.transaction_id) as qtd_transacoes,
            SUM(t.amount) as total_gasto,
            AVG(t.amount) as media_gasto
        FROM transactions t
        JOIN categories c ON t.category_id = c.category_id
        WHERE t.user_phone = '5511999999999'
        GROUP BY c.category_name
        ORDER BY total_gasto DESC
    """).fetchall()
    
    for row in gastos:
        cat, qtd, total, media = row
        print(f"{cat:15s} | {qtd} transações | Total: R$ {total:7.2f} | Média: R$ {media:6.2f}")
    
    # Status dos limites
    print("\n⚠️  STATUS DOS LIMITES")
    print("-" * 60)
    limites = conn.execute("""
        SELECT 
            c.category_name,
            r.limit_value,
            r.current_total,
            ROUND((r.current_total * 100.0 / r.limit_value), 2) as percentual
        FROM user_rules r
        JOIN categories c ON r.category_id = c.category_id
        WHERE r.user_phone = '5511999999999' AND r.active = 1
    """).fetchall()
    
    for row in limites:
        cat, limite, atual, perc = row
        status = "🔴 EXCEDIDO" if perc > 100 else "🟡 ATENÇÃO" if perc > 80 else "🟢 OK"
        print(f"{cat:15s} | {atual:7.2f} / {limite:7.2f} ({perc:5.1f}%) {status}")


📊 ESTATÍSTICAS GERAIS
------------------------------------------------------------
Total de usuários: 3
Total de categorias: 6
Total de transações: 8
Total de regras: 3

💰 GASTOS POR CATEGORIA (João Silva)
------------------------------------------------------------
Alimentação     | 2 transações | Total: R$  166.40 | Média: R$  83.20
Lazer           | 1 transações | Total: R$   80.00 | Média: R$  80.00
Transporte      | 2 transações | Total: R$   43.50 | Média: R$  21.75

⚠️  STATUS DOS LIMITES
------------------------------------------------------------
Alimentação     |  166.40 /  500.00 ( 33.3%) 🟢 OK
Transporte      |   43.50 /  150.00 ( 29.0%) 🟢 OK
Lazer           |   80.00 /  300.00 ( 26.7%) 🟢 OK


## 🧹 Limpeza (Opcional)

Se quiser remover os dados de teste e começar do zero, execute a célula abaixo:


In [15]:
# Descomente as linhas abaixo para limpar os dados de teste

# with get_connection() as conn:
#     conn.execute("DELETE FROM user_rules")
#     conn.execute("DELETE FROM transactions")
#     conn.execute("DELETE FROM categories")
#     conn.execute("DELETE FROM users")
#     conn.commit()
#     print("✅ Dados de teste removidos!")


In [16]:
from database import save_message, get_connection

# grava algumas mensagens de teste
save_message("5511999999999", "Gastei 40 reais no mercado")
save_message("5511999999999", "Uber residência → trabalho 25,90")
save_message("5511888888888", "Cinema 50,00")

with get_connection() as conn:
    rows = conn.execute("""
        SELECT transaction_id, user_phone, amount, expense_description, created_at
        FROM transactions
        ORDER BY transaction_id DESC
        LIMIT 5;
    """).fetchall()

for row in rows:
    print(dict(row))

{'transaction_id': 11, 'user_phone': '5511888888888', 'amount': 0.0, 'expense_description': 'Cinema 50,00', 'created_at': '2025-11-12 23:05:44'}
{'transaction_id': 10, 'user_phone': '5511999999999', 'amount': 0.0, 'expense_description': 'Uber residência → trabalho 25,90', 'created_at': '2025-11-12 23:05:44'}
{'transaction_id': 9, 'user_phone': '5511999999999', 'amount': 0.0, 'expense_description': 'Gastei 40 reais no mercado', 'created_at': '2025-11-12 23:05:44'}
{'transaction_id': 8, 'user_phone': '5511999999999', 'amount': 80.0, 'expense_description': 'Cinema com amigos', 'created_at': '2025-11-09 20:05:35.270698'}
{'transaction_id': 7, 'user_phone': '5511999999999', 'amount': 18.5, 'expense_description': 'Uber voltando', 'created_at': '2025-11-12 18:05:35.270697'}
